# SWELL - stress dataset

In [ ]:
import os
import time
# import shap ## for XAI
import numpy as np
import pandas as pd
import seaborn as sns
# import pingouin as pg
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [ ]:
from numpy import dot
from numpy.linalg import norm

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.layers import Dense , Activation, Dropout, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.callbacks import Callback

from scipy.special import rel_entr
from scipy.stats import gaussian_kde
from scipy.spatial.distance import jensenshannon

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, train_test_split, ParameterGrid
from sklearn import decomposition, metrics
from sklearn.cluster import KMeans
from sklearn.metrics import cohen_kappa_score,f1_score, confusion_matrix, roc_auc_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import Normalizer, MinMaxScaler, RobustScaler
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.impute import KNNImputer
from sklearn.manifold import TSNE

from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Lambda, Conv1D, Flatten, Reshape, UpSampling1D
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models, losses, backend as K
from tensorflow.keras.losses import mse, MeanSquaredError

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import tensorflow as tf 
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

## Data Check

In [ ]:
data_ori = pd.read_csv('E:/RESEARCH/Datasets/bio_data/hrv_eda/2. final/datasets/hrv/swell/combined/classification/swell_hrv.csv')

In [ ]:
### data shape, variables check
print("The shape of the original SWELL HRV dataset is:", data_ori.shape)
print(data_ori.columns)
# data_ori.head()

> Target label is "condition" variable. \
> no stress,  time pressur and = interruption

* Labeling check

In [ ]:
data_ori["condition"].value_counts()

***

### NA handling

> Drop only rows with NA in target_info column. \
> Data row without diagnosed target_info is difficult to use.

In [ ]:
print("Data row that has no target information is:", data_ori['condition'].isna().sum())

In [ ]:
data_ori = data_ori.dropna(subset = ['condition'], how='any', axis=0)

In [ ]:
data_ori.shape

***

### Reducing dataset size

In [ ]:
# df_reduced = data_ori.groupby('condition').head(3000)

In [ ]:
df_reduced = data_ori.groupby('condition').apply(lambda x: x.sample(500)).reset_index(drop=True)

In [ ]:
df_reduced['condition'].value_counts()

---

### JSD for meaningful variable extraction

In [ ]:
data_vis = df_reduced.copy()

In [ ]:
data_vis = data_vis.drop(['condition'], axis=1)
features = [col for col in list(data_vis.columns)]

In [ ]:
# def jsd_multiple_distributions(*distributions):
#     num_distributions = len(distributions)
#     average_distribution = np.mean(distributions, axis=0)
#     jsd = 0
#     for dist in distributions:
#         jsd += jensenshannon(dist, average_distribution) ** 2
#     jsd /= num_distributions
#     return np.sqrt(jsd)

In [ ]:
## list to append distributional differences
variable = []
diff01 = []
diff02 = []
diff03 = []
diff12 = []
diff13 = []
diff23 = []
# diff123 = []
# diff0123 = []

for feature in features:
    ## deleting any NA values in each feature column, for distance calculation
    data = data_vis.dropna(subset=[feature])

    ## split the dataset into each classes
    df0 = data[data['target_info']=="control"]
    df1 = data[data['target_info']=="mdd"]
    df2 = data[data['target_info']=="bpi"]
    df3 = data[data['target_info']=="bpii"]

    df0 = df0.drop(['target_info'], axis=1)
    df1 = df1.drop(['target_info'], axis=1)
    df2 = df2.drop(['target_info'], axis=1)
    df3 = df3.drop(['target_info'], axis=1)
    
    df0_values = df0[feature]
    df1_values = df1[feature]
    df2_values = df2[feature]
    df3_values = df3[feature]

    ## sampling based on the minimum size of the target info.
    sample_size = (min(df0_values.shape[0], df1_values.shape[0], df2_values.shape[0], df3_values.shape[0]))
    df0_sample = df0_values.sample(n=(sample_size))
    df1_sample = df1_values.sample(n=(sample_size))
    df2_sample = df2_values.sample(n=(sample_size))
    df3_sample = df3_values.sample(n=(sample_size))

    ## calculating distribution using histogram
    hist0, bin_edges0 = np.histogram(df0_sample, bins=30, density=True)
    hist1, bin_edges1 = np.histogram(df1_sample, bins=30, density=True)
    hist2, bin_edges2 = np.histogram(df2_sample, bins=30, density=True)
    hist3, bin_edges3 = np.histogram(df3_sample, bins=30, density=True)

    ## standardize the probability distribution
    h0 = hist0 / np.sum(hist0)
    h1 = hist1 / np.sum(hist1)
    h2 = hist2 / np.sum(hist2)
    h3 = hist3 / np.sum(hist3)

    ## jensen-shannon divergence calculation and append
    variable.append(feature)
    diff01.append(jensenshannon(h0, h1))
    diff02.append(jensenshannon(h0, h2))
    diff03.append(jensenshannon(h0, h3))
    diff12.append(jensenshannon(h1, h2))
    diff13.append(jensenshannon(h1, h3))
    diff23.append(jensenshannon(h2, h3))
    # diff123.append(jsd_multiple_distributions(h1, h2, h3))
    # diff0123.append(jsd_multiple_distributions(h0, h1, h2, h3))

In [ ]:
jsd_differences = pd.DataFrame({'JSD01':diff01, 'JSD02':diff02, 'JSD03':diff03, 'JSD12':diff12, 'JSD13':diff13, 'JSD23':diff23})
# jsd_differences = pd.DataFrame({'variable':variable, 'JSD01':diff01, 'JSD02':diff02, 'JSD03':diff03, 'JSD12':diff12, 'JSD13':diff13, 'JSD23':diff23})
# jsd_differences = pd.DataFrame({'variable':variable, 'JSD01':diff01, 'JSD02':diff02, 'JSD03':diff03, 'JSD12':diff12, 'JSD13':diff13, 'JSD23':diff23, 
#                                'JSD123':diff123, 'JSD0123':diff0123})

In [ ]:
jsd_differences.head()

In [ ]:
# definition to check for all row, whether one or more value is bigger than 0.2
def check_jsd(row):
    return 'yes' if any(row >= 0.3) else 'no'

# using apply method to create new column 'result'
jsd_differences['result'] = jsd_differences.apply(check_jsd, axis=1)

In [ ]:
jsd_differences['mean'] = jsd_differences[['JSD01','JSD02','JSD03','JSD12','JSD13','JSD23']].mean(axis=1)

In [ ]:
jsd_differences.head()

In [ ]:
jsd_differences['variable'] = variable

In [ ]:
# jsd_big_diff = jsd_differences[jsd_differences['mean']>0.3]
# jsd_big_diff = jsd_differences[jsd_differences['JSD123']>0.3]
jsd_big_diff = jsd_differences[jsd_differences['result']=='yes']

In [ ]:
big_diff_columns = jsd_big_diff['variable']

In [ ]:
big_diff_columns = jsd_big_diff['variable'].values.tolist()

In [ ]:
# print(big_diff_columns)

In [ ]:
len(big_diff_columns)

In [ ]:
len(data_vis.columns)

> We can check that there are 27 variables lacks distributional distance between target groups

***

---

## Support Vector Machine

In [ ]:
data = data_fill_s8.copy()

In [ ]:
data_x = data.drop(['subject_info', 'birth_info', 'target_info', 'sex_info', 'age_info', 'family_hx_info'], axis=1)
data_y = data.loc[:,["target_info"]]

In [ ]:
## this is for selecting variables with JSDivergence big differences
data_x = data_x[big_diff_columns]

In [ ]:
data_x = data_x.fillna(data_x.mean())

In [ ]:
########## FULL GROUP CLASSIFICATION ##########
label = data_y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

# y = to_categorical(label, 4) ## into the format of one-hot encoding
y = label

In [ ]:
x = data_x

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 710674)

* Support vector machine algorithm

In [ ]:
from sklearn import svm

svm_correct = []
for i in range(250):
    clf = svm.SVC(random_state=710674, 
                  #class_weight=rf_class_w_dict,
                  kernel ='poly',
                  max_iter=i,
                  verbose=3
                 )    
    clf.fit(x_train, y_train)
    predictions = clf.predict(x_test).reshape(x_test.shape[0],1)
    test_correct = (predictions == y_test, 1)[0].sum().item()
    test_correct /= len(y_test)

    print(test_correct)
    svm_correct.append(test_correct)

In [ ]:
max(svm_correct)

***

## Normal DNN

In [ ]:
data = df_reduced

In [ ]:
data.columns

* Deleting all NA values

In [ ]:
class Args:
    # arugments
    epochs=200
    bs=64
    lr=0.0001
    momentum=0.9
    num_classes= 3
    verbose='store_true'
    seed=710674

args = Args()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
data_x = data.drop(['condition'], axis=1)
data_y = data.loc[:,["condition"]]

In [ ]:
## this is for selecting variables with JSDivergence big differences
# data_x = data_x[big_diff_columns]

## this is for selecting variables without JSDivergence big differences
# data_x = data_x.drop(big_diff_columns, axis=1)

* Filling NA values with mean values

In [ ]:
data_y.value_counts()

In [ ]:
########## FULL GROUP CLASSIFICATION ##########
label = data_y
label = label.replace({'interruption':0})
label = label.replace({'no stress': 1})
label = label.replace({'time pressure': 2})

y = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
x = data_x

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)
# scaler = RobustScaler()
x[:] = scaler.fit_transform(x[:])
x = x.round(decimals=6)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 710674)

In [ ]:
print("The size of training dataset is:", x_train.shape[0])
print("The size of test dataset is:", x_test.shape[0])

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
class_weight = {0:1, 1:1, 2:1}

In [ ]:
data_x.shape

In [ ]:
#### Generate the model
## this model is adequate for full variables usage
model = Sequential()
model.add(Dense(64, input_dim = x_train.shape[1], activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(args.num_classes, activation = 'softmax'))

***

In [ ]:
def cyclic_learning_rate(epoch, base_lr=0.001, max_lr=0.006, step_size=2000):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x))
    return lr

In [ ]:
def triangular_lr(epoch, base_lr=0.001, max_lr=0.006, step_size=2000):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x))
    return lr

In [ ]:
def triangular2_lr(epoch, base_lr=0.001, max_lr=0.006, step_size=2000):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x)) / (2 ** (cycle - 1))
    return lr

In [ ]:
def exp_range_lr(epoch, base_lr=0.001, max_lr=0.006, step_size=2000, gamma=0.99994):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * max(0, (1 - x)) * (gamma ** epoch)
    return lr

* optimization function, model compile, and model training

In [ ]:
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-5, momentum = args.momentum)

In [ ]:
# LearningRateScheduler 설정
# lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=300))
lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=30, gamma=0.99994))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2)   ## using no class weight
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2, class_weight = class_weight)  ## using class weight
history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, callbacks=[lr_scheduler], class_weight = class_weight, verbose=2) ## using LR scheduler callback

***

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, model.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

## k-fold DNN

In [ ]:
# data = data_no_s8.copy()
# data = data_fill_s8.copy()
# data = data_nona.copy()
# data = data_fill.copy()
# data = data_fill_mean.copy()
# data = data_fill_mean_age.copy()
data = data_fill_mean_knn.copy()

In [ ]:
data_x = data.drop(['subject_info', 'birth_info', 'target_info', 'sex_info', 'age_info', 'family_hx_info', 'marriage_info', 'job_info'], axis=1)
data_y = data.loc[:,["target_info"]]

In [ ]:
# ## this is for selecting variables with JSDivergence big differences
# data_x = data_x[big_diff_columns]

# ## this is for selecting variables without JSDivergence big differences
# # data_x = data_x.drop(big_diff_columns, axis=1)

In [ ]:
data_x = data_x.fillna(data_x.mean())

In [ ]:
########## FULL GROUP CLASSIFICATION ##########
label = data_y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

y = to_categorical(label, 4) ## into the format of one-hot encoding

In [ ]:
x = data_x

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(data_x, y, test_size = 0.2, random_state = 710674)

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_trainset, y_trainset, test_size = 0.1, random_state = 710674)

In [ ]:
x_train.shape

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
kfold = KFold(n_splits = split_num, shuffle = True)
# kfold = StratifiedKFold(n_splits = split_num, shuffle = True)

In [ ]:
# LearningRateScheduler 설정
lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=300))
# lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=300, gamma=0.99994))

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
# class_weight = {0:1, 1:1.29, 2:3.04, 3: 1.13}
class_weight = {0:1, 1:1.81, 2:4.63, 3: 1.44}
# class_weight = {0:1.15, 1:2.09, 2:2.84, 3: 1}

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, callbacks=[lr_scheduler],
                        verbose = 0, class_weight = class_weight)
    # history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
# for train, test in kfold.split(inputs, targets):
#     model = Sequential()
#     model.add(Dense(512, input_dim = x_train.shape[1], activation = 'relu'))
#     model.add(Dense(1024, activation = 'relu'))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(Dense(512, activation = 'relu'))
#     model.add(BatchNormalization())
#     model.add(Activation('relu'))
#     model.add(Dense(256, activation = 'relu'))
#     model.add(Dense(64, activation = 'relu'))
#     model.add(Dense(32, activation = 'relu'))
#     model.add(Dense(args.num_classes, activation = 'softmax'))
    
#     ## model compile
#     model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
#     print('----------------------------------------')
#     print(f'Training or fold {fold_num} ... ')
    
#     ## fit data to model
#     history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, callbacks=[lr_scheduler],
#                         verbose = 0, class_weight = class_weight)
#     # history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    
#     ## generate generalization metrics
#     scores = model.evaluate(inputs[test], targets[test])
#     print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
#     print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
#     acc_per_fold.append(scores[1] * 100)
#     loss_per_fold.append(scores[0])
    
#     ## increasing fold number
#     fold_num = fold_num + 1
    
    
    
# ## Summarizing the results
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#     print('------------------------------------------------------------------------')
#     print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'>>> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, model.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

## Multimodal DNN

In [ ]:
class Args_multi:
    # arugments
    epochs=150
    enc_epochs = 50
    bs=32
    enc_bs = 16
    lr=0.0001
    momentum=0.9
    num_classes= 4
    seed=710674

args_multi = Args_multi()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
# data = data_no_s8.copy()
# data = data_fill_s8.copy()
# data = data_nona.copy()
# data = data_fill.copy()
# data = data_fill_mean.copy()
# data = data_fill_mean_age.copy()
data = data_fill_mean_knn.copy()

In [ ]:
data['target_info'].value_counts()

In [ ]:
data.columns

In [ ]:
data_x = data.drop(['subject_info', 'birth_info', 'target_info', 'sex_info', 'age_info', 'family_hx_info', 'marriage_info', 'job_info'], axis=1)
data_y = data.loc[:,["target_info"]]

In [ ]:
# ## this is for selecting variables with JSDivergence big differences
# data_x = data_x[big_diff_columns]

# ## this is for selecting variables without JSDivergence big differences
# # data_x = data_x.drop(big_diff_columns, axis=1)

In [ ]:
data_x = data_x.fillna(data_x.mean())

In [ ]:
########## FULL GROUP CLASSIFICATION ##########
label = data_y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

y = to_categorical(label, 4) ## into the format of one-hot encoding

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
# class_weight = {0:1, 1:1.29, 2:3.04, 3: 1.13}
class_weight = {0:1, 1:1.81, 2:4.63, 3: 1.44}
# class_weight = {0:1.15, 1:2.09, 2:2.84, 3: 1}

### using subnetworks

* dataframe contains section 2,4,5,6,7,8

In [ ]:
data_s2 = data_x.iloc[:, (data_x.columns.str.contains(('s2')))]
data_s4 = data_x.iloc[:, (data_x.columns.str.contains(('s4')))]
data_s5 = data_x.iloc[:, (data_x.columns.str.contains(('s5')))]
data_s6 = data_x.iloc[:, (data_x.columns.str.contains(('s6')))]
data_s7 = data_x.iloc[:, (data_x.columns.str.contains(('s7')))]
data_s8 = data_x.iloc[:, (data_x.columns.str.contains(('s8')))]

In [ ]:
# data_s5.columns

In [ ]:
# 입력 정의
input2 = Input(shape=(data_s2.shape[1],))
input4 = Input(shape=(data_s4.shape[1],))
input5 = Input(shape=(data_s5.shape[1],))
input6 = Input(shape=(data_s6.shape[1],))
input7 = Input(shape=(data_s7.shape[1],))
input8 = Input(shape=(data_s8.shape[1],))

---

* Creating commonly using sub-network

In [ ]:
# 서브 네트워크 정의 함수
def create_subnetwork(input_layer):
    x = Dense(32, activation='relu')(input_layer)
    # x = Dense(16, activation='relu')(x)
    x = BatchNormalization()(x)
    # x = Dropout(0.3)(x)
    x = Dense(8, activation='relu')(x)
    return x

In [ ]:
# 서브 네트워크 생성
subnet2 = create_subnetwork(input2)
subnet4 = create_subnetwork(input4)
subnet5 = create_subnetwork(input5)
subnet6 = create_subnetwork(input6)
subnet7 = create_subnetwork(input7)
subnet8 = create_subnetwork(input8)

=======================================================================================================

In [ ]:
input8

In [ ]:
# 서브 네트워크 정의 함수
def create_subnetwork2(input_layer):
    x = Dense(32, activation='relu')(input_layer)
    x = Dense(8, activation='relu')(x)
    return x

def create_subnetwork4(input_layer):
    x = Dense(16, activation='relu')(input_layer)
    x = Dense(8, activation='relu')(x)
    return x

def create_subnetwork5(input_layer):
    x = Dense(32, activation='relu')(input_layer)
    # x = Dense(32, activation='relu')(x)
    x = Dense(8, activation='relu')(x)
    return x

def create_subnetwork6(input_layer):
    x = Dense(32, activation='relu')(input_layer)
    # x = Dense(64, activation='relu')(x)
    # x = Dense(32, activation='relu')(x)
    x = Dense(8, activation='relu')(x)
    return x

def create_subnetwork7(input_layer):
    x = Dense(32, activation='relu')(input_layer)
    # x = Dense(32, activation='relu')(x)
    x = Dense(8, activation='relu')(x)
    return x

def create_subnetwork8(input_layer):
    x = Dense(16, activation='relu')(input_layer)
    x = Dense(8, activation='relu')(x)
    return x

In [ ]:
# 서브 네트워크 생성
subnet2 = create_subnetwork2(input2)
subnet4 = create_subnetwork4(input4)
subnet5 = create_subnetwork5(input5)
subnet6 = create_subnetwork6(input6)
subnet7 = create_subnetwork7(input7)
subnet8 = create_subnetwork8(input8)

***

In [ ]:
# 서브 네트워크 출력 결합
# combined = Concatenate()([subnet2, subnet4, subnet5, subnet6, subnet7])
combined = Concatenate()([subnet2, subnet4, subnet5, subnet6, subnet7, subnet8])

In [ ]:
combined

In [ ]:
## adding dense(fc) layers after encoder combining
concat_x = Dense(32, activation='relu')(combined)
concat_x = BatchNormalization()(concat_x)
concat_x = Dropout(0.5)(concat_x)
# concat_x = Dense(16, activation='relu')(concat_x)
# concat_x = BatchNormalization()(concat_x)
# concat_x = Dropout(0.5)(concat_x)
# concat_x = Dense(16, activation='relu')(concat_x)
concat_x = Dense(8, activation='relu')(concat_x)

# 최종 출력 레이어 정의 (4-class 분류)
final_output = Dense(args_multi.num_classes, activation='softmax')(concat_x)

In [ ]:
# # 최종 출력 레이어 정의
# final_output = Dense(args_multi.num_classes, activation='softmax')(combined) 

In [ ]:
# x2_train, x2_test, x4_train, x4_test, x5_train, x5_test, x6_train, x6_test, x7_train, x7_test, y_train, y_test = train_test_split(
#     data_s2, data_s4, data_s5, data_s6, data_s7, y, test_size=0.2, random_state=710674)

x2_train, x2_test, x4_train, x4_test, x5_train, x5_test, x6_train, x6_test, x7_train, x7_test, x8_train, x8_test, y_train, y_test = train_test_split(
    data_s2, data_s4, data_s5, data_s6, data_s7, data_s8, y, test_size=0.2, random_state=710674)

In [ ]:
# LearningRateScheduler 설정
lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=300))
# lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=300, gamma=0.99994))

In [ ]:
opt = keras.optimizers.SGD(learning_rate = args_multi.lr, decay = 1e-6, momentum = args_multi.momentum)

In [ ]:
# 모델 생성 및 컴파일
# model = Model(inputs=[input2, input4, input5, input6, input7], outputs=final_output)
model = Model(inputs=[input2, input4, input5, input6, input7, input8], outputs=final_output)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# model.fit([data_s2, data_s4, data_s5, data_s6, data_s7, data_s8], y, 
#           epochs=args_multi.epochs, batch_size=args_multi.bs, verbose=2)

# model.fit([x2_train, x4_train, x5_train, x6_train, x7_train, x8_train], y_train, 
#           epochs=args_multi.epochs, batch_size=args_multi.bs, verbose=2)


#########################################

# model.fit([x2_train, x4_train, x5_train, x6_train, x7_train], y_train, 
#           epochs=args_multi.epochs, batch_size=args_multi.bs, callbacks=[lr_scheduler], verbose=2)
model.fit([x2_train, x4_train, x5_train, x6_train, x7_train, x8_train], y_train, 
          epochs=args_multi.epochs, batch_size=args_multi.bs, callbacks=[lr_scheduler], verbose=2)

In [ ]:
# y_predict = model.predict([x2_test, x4_test, x5_test, x6_test, x7_test])
y_predict = model.predict([x2_test, x4_test, x5_test, x6_test, x7_test, x8_test])

y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
# auc = roc_auc_score(y_test,model.predict([x2_test, x4_test, x5_test, x6_test, x7_test], verbose=0), multi_class='ovr')
auc = roc_auc_score(y_test,model.predict([x2_test, x4_test, x5_test, x6_test, x7_test, x8_test], verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

***

### using individual autoencoder

In [ ]:
x2_train, x2_test, x4_train, x4_test, x5_train, x5_test, x6_train, x6_test, x7_train, x7_test, x8_train, x8_test, y_train, y_test = train_test_split(
    data_s2, data_s4, data_s5, data_s6, data_s7, data_s8, y, test_size=0.2, random_state=710674)

***

* Using same autoencoder structure for all input datasets

In [ ]:
## Defining autoencoder
def create_autoencoder(input_shape):
    # defining encoder model
    input_layer = Input(shape=input_shape)
    encoded = Dense(32, activation='relu')(input_layer)
    # encoded = Dense(16, activation='relu')(encoded)
    encoded = Dense(8, activation='relu')(encoded)
    
    # defining decoder model
    # decoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    
    # setting autoencoder model
    autoencoder = Model(input_layer, decoded)
    
    # extracting encoder model separately
    encoder = Model(input_layer, encoded)
    
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder, encoder

In [ ]:
# Defining input dataset shape
input_shape2 = (data_s2.shape[1],)
input_shape4 = (data_s4.shape[1],)
input_shape5 = (data_s5.shape[1],)
input_shape6 = (data_s6.shape[1],)
input_shape7 = (data_s7.shape[1],)
input_shape8 = (data_s8.shape[1],)

In [ ]:
autoencoder2, encoder2 = create_autoencoder(input_shape2)
autoencoder4, encoder4 = create_autoencoder(input_shape4)
autoencoder5, encoder5 = create_autoencoder(input_shape5)
autoencoder6, encoder6 = create_autoencoder(input_shape6)
autoencoder7, encoder7 = create_autoencoder(input_shape7)
autoencoder8, encoder8 = create_autoencoder(input_shape8)

==========================================

* Using individually difference structured autoencoder model

In [ ]:
# Defining input dataset shape
input_shape2 = (data_s2.shape[1],)
input_shape4 = (data_s4.shape[1],)
input_shape5 = (data_s5.shape[1],)
input_shape6 = (data_s6.shape[1],)
input_shape7 = (data_s7.shape[1],)
input_shape8 = (data_s8.shape[1],)

In [ ]:
input_shape8

In [ ]:
## Defining autoencoder
def create_autoencoder2(input_shape):
    # defining encoder model
    input_layer = Input(shape=input_shape)
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

def create_autoencoder4(input_shape):
    # defining encoder model
    input_layer = Input(shape=input_shape)
    encoded = Dense(16, activation='relu')(input_layer)
    encoded = Dense(4, activation='relu')(encoded)
    decoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

def create_autoencoder5(input_shape):
    # defining encoder model
    input_layer = Input(shape=input_shape)
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

def create_autoencoder6(input_shape):
    # defining encoder model
    input_layer = Input(shape=input_shape)
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

def create_autoencoder7(input_shape):
    # defining encoder model
    input_layer = Input(shape=input_shape)
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

def create_autoencoder8(input_shape):
    # defining encoder model
    input_layer = Input(shape=input_shape)
    encoded = Dense(16, activation='relu')(input_layer)
    encoded = Dense(4, activation='relu')(encoded)
    decoded = Dense(16, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

In [ ]:
autoencoder2, encoder2 = create_autoencoder2(input_shape2)
autoencoder4, encoder4 = create_autoencoder4(input_shape4)
autoencoder5, encoder5 = create_autoencoder5(input_shape5)
autoencoder6, encoder6 = create_autoencoder6(input_shape6)
autoencoder7, encoder7 = create_autoencoder7(input_shape7)
autoencoder8, encoder8 = create_autoencoder8(input_shape8)

***

In [ ]:
# Training autoencoder model
# autoencoder2.fit(x2_train, x2_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs)
# autoencoder4.fit(x4_train, x4_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs)
# autoencoder5.fit(x5_train, x5_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs)
# autoencoder6.fit(x6_train, x6_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs)
# autoencoder7.fit(x7_train, x7_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs)
# autoencoder8.fit(x8_train, x8_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs)

# Training autoencoder model
autoencoder2.fit(x2_train, x2_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs, validation_split=0.2)
autoencoder4.fit(x4_train, x4_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs, validation_split=0.2)
autoencoder5.fit(x5_train, x5_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs, validation_split=0.2)
autoencoder6.fit(x6_train, x6_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs, validation_split=0.2)
autoencoder7.fit(x7_train, x7_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs, validation_split=0.2)
autoencoder8.fit(x8_train, x8_train, epochs=args_multi.enc_epochs, batch_size=args_multi.enc_bs, validation_split=0.2)

In [ ]:
# Defining input
input2 = Input(shape=(data_s2.shape[1],))
input4 = Input(shape=(data_s4.shape[1],))
input5 = Input(shape=(data_s5.shape[1],))
input6 = Input(shape=(data_s6.shape[1],))
input7 = Input(shape=(data_s7.shape[1],))
input8 = Input(shape=(data_s8.shape[1],))

In [ ]:
# Defining subnetwork function
def create_encoded_output(input_layer, encoder):
    return encoder(input_layer)

In [ ]:
# 서브 네트워크 생성
encoded2 = create_encoded_output(input2, encoder2)
encoded4 = create_encoded_output(input4, encoder4)
encoded5 = create_encoded_output(input5, encoder5)
encoded6 = create_encoded_output(input6, encoder6)
encoded7 = create_encoded_output(input7, encoder7)
encoded8 = create_encoded_output(input8, encoder8)

In [ ]:
# 서브 네트워크 출력 결합
combined = Concatenate()([encoded2, encoded4, encoded5, encoded6, encoded7, encoded8])

In [ ]:
combined

In [ ]:
## adding dense(fc) layers after encoder combining
concat_x = Dense(32, activation='relu')(combined)
# concat_x = Dense(64, activation='relu')(concat_x)
concat_x = BatchNormalization()(concat_x)
concat_x = Dropout(0.5)(concat_x)
# concat_x = Dense(32, activation='relu')(concat_x)
# concat_x = Dropout(0.2)(concat_x)
concat_x = Dense(8, activation='relu')(concat_x)

# 최종 출력 레이어 정의 (4-class 분류)
final_output = Dense(args_multi.num_classes, activation='softmax')(concat_x)

In [ ]:
# 모델 생성 및 컴파일
model = Model(inputs=[input2, input4, input5, input6, input7, input8], outputs=final_output)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

In [ ]:
# LearningRateScheduler 설정
lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=300))
# lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=300, gamma=0.99994))

In [ ]:
## Model training
# model.fit([x2_train, x4_train, x5_train, x6_train, x7_train, x8_train], y_train, epochs=args_multi.epochs, batch_size=args_multi.bs)
model.fit([x2_train, x4_train, x5_train, x6_train, x7_train, x8_train], y_train, epochs=args_multi.epochs, callbacks = [lr_scheduler], batch_size=args_multi.bs)

## Model evaluation
# loss, accuracy = model.evaluate([x2_test, x4_test, x5_test, x6_test, x7_test, x8_test], y_test)
# print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
y_predict = model.predict([x2_test, x4_test, x5_test, x6_test, x7_test, x8_test])
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test,model.predict([x2_test, x4_test, x5_test, x6_test, x7_test, x8_test], verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

## VAE augmentation

In [ ]:
data = df_reduced.copy()

> We sampled 300 data for each label, to augment other 100 data for each class. \
> This is to compare the classification performance of data augmentation.

In [ ]:
data = data.groupby('condition').apply(lambda x: x.sample(400)).reset_index(drop=True)

In [ ]:
data["condition"].value_counts()

In [ ]:
data_int = data[data['condition']=="interruption"]
data_nos = data[data['condition']=="no stress"]
data_tip = data[data['condition']=="time pressure"]

In [ ]:
class Args_aug:
    # arugments
    epochs=150
    bs=32
    lr=0.0001
    momentum=0.9
    num_classes= 2
    latent_dim = 16
    inter_dim1 = 32
    inter_dim2 = 16
    seed=710674

args_aug = Args_aug()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
# data_vae = data.copy()
# data_vae = data_int.copy()
# data_vae = data_nos.copy()
data_vae = data_tip.copy()

In [ ]:
data_vae.columns

In [ ]:
# y includes our labels and x includes our features
y = data_vae.condition  # M or B 
list = ['condition']
x = data_vae.drop(list, axis = 1 )

In [ ]:
data_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

data_x[:] = scaler.fit_transform(data_x[:])
data_x = data_x.round(decimals=6)

In [ ]:
# # 데이터 확인
# print(np.isnan(data_x).any())  # False여야 합니다.
# print(np.isinf(data_x).any())  # False여야 합니다.

In [ ]:
# data_x = data_x.fillna(data_x.mean())

In [ ]:
########## FULL GROUP CLASSIFICATION ##########
label = y
label = label.replace({'interruption':0})
label = label.replace({'no stress': 1})
label = label.replace({'time pressure': 2})

data_y = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
print("The size of x dataset is:", data_x.shape)
print("The size of y dataset is:", data_y.shape)

### encoder networks

In [ ]:
# ## shallow model
# input_dim = data_x.shape[1]
# latent_dim = 2

# inputs = Input(shape=(input_dim,))
# h = Dense(32, activation='relu')(inputs)
# z_mean = Dense(latent_dim)(h)
# z_log_var = Dense(latent_dim)(h)

In [ ]:
## deeper model
input_dim = data_x.shape[1]
latent_dim = 8

inputs = Input(shape=(input_dim,))
h = Dense(64, activation='relu')(inputs)
h = Dense(32, activation='relu')(h)
h = Dense(16, activation='relu')(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [ ]:
## latent space sampling
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

### decoder networks

In [ ]:
# ## shallow model
# decoder_h = Dense(32, activation='relu')
# decoder_mean = Dense(input_dim, activation='sigmoid')

# h_decoded = decoder_h(z)
# x_decoded_mean = decoder_mean(h_decoded)

In [ ]:
## deeper model
decoder_h1 = Dense(16, activation='relu')
decoder_h2 = Dense(32, activation='relu')
decoder_h3 = Dense(64, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')

h_decoded = decoder_h1(z)
h_decoded = decoder_h2(h_decoded)
h_decoded = decoder_h3(h_decoded)
x_decoded_mean = decoder_mean(h_decoded)

### model define

In [ ]:
vae = Model(inputs, x_decoded_mean)
vae.summary()

### loss function

In [ ]:
reconstruction_loss = MeanSquaredError()(inputs, x_decoded_mean)

kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var + K.epsilon()), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

In [ ]:
vae.compile(optimizer=Adam(learning_rate=args_aug.lr))

In [ ]:
vae.summary()

### model training

In [ ]:
vae.fit(data_x, epochs=args_aug.epochs, batch_size=args_aug.bs, validation_split=0.2, verbose=2)

### extracting encoder, decoder

In [ ]:
## encoder model extraction
encoder = Model(inputs, z_mean)

In [ ]:
# ## decoder model extraction
# ## shallow model
# decoder_input = Input(shape=(latent_dim,))
# _h_decoded = decoder_h(decoder_input)
# _x_decoded_mean = decoder_mean(_h_decoded)
# decoder = Model(decoder_input, _x_decoded_mean)

## deeper model
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h1(decoder_input)
_h_decoded = decoder_h2(_h_decoded)
_h_decoded = decoder_h3(_h_decoded)
_x_decoded_mean = decoder_mean(_h_decoded)
decoder = Model(decoder_input, _x_decoded_mean)

In [ ]:
# # 입력 데이터를 잠재 공간으로 인코딩
# encoded_data = encoder.predict(data_x)

# # 잠재 공간의 데이터를 디코딩하여 원래 공간으로 복원
# decoded_data = decoder.predict(encoded_data)

### Latent space visualization

In [ ]:
## encode the input dataset into latent space
encoded_data = encoder.predict(data_x)

## latent space transform with t-SNE function
tsne = TSNE(n_components=2, random_state=710674)
encoded_data_tsne = tsne.fit_transform(encoded_data)

## visualize
plt.figure(figsize=(8, 6))
plt.scatter(encoded_data_tsne[:, 0], encoded_data_tsne[:, 1], c=label, cmap='viridis')
plt.colorbar()
plt.xlabel("t-SNE component 1")
plt.ylabel("t-SNE component 2")
plt.title("t-SNE visualization of the latent space")
plt.show()

### Synthetic data generation

In [ ]:
### sampling randomly from latent space
n_samples = 100  # number of synthetic dataset to generate

z_samples = np.random.normal(size=(n_samples, latent_dim))

### 디코더를 통해 synthetic data 생성
# synthetic_data_int = decoder.predict(z_samples)
# synthetic_data_nos = decoder.predict(z_samples)
synthetic_data_tip = decoder.predict(z_samples)

In [ ]:
# gen_int = synthetic_data_int.copy()
# gen_int = pd.DataFrame(gen_int, columns=data_x.columns)

# #####################
# gen_nos = synthetic_data_nos.copy()
# gen_nos = pd.DataFrame(gen_nos, columns=data_x.columns)

#####################
gen_tip = synthetic_data_tip.copy()
gen_tip = pd.DataFrame(gen_tip, columns=data_x.columns)

### performance comparison

#### using original + generated dataset

* Set target information into newly generated dataset

In [ ]:
gen_int['condition'] = "interruption"
gen_nos['condition'] = "no stress"
gen_tip['condition'] = "time pressure"

* Preparing original dataset

In [ ]:
data.shape

In [ ]:
data_int = data[data['condition']=="interruption"]
data_nos = data[data['condition']=="no stress"]
data_tip = data[data['condition']=="time pressure"]

* Concat the original datasets into one original dataframe

In [ ]:
ori_df_list = [data_int, data_nos, data_tip]
ori_df_concat = pd.concat(ori_df_list, ignore_index=True)

In [ ]:
ori_df_concat.shape

In [ ]:
ori_df_concat['condition'].value_counts()

* Concat the generated datasets into one gen dataframe

In [ ]:
gen_df_list = [gen_int, gen_nos, gen_tip]
gen_df_concat = pd.concat(gen_df_list, ignore_index=True)

In [ ]:
gen_df_concat['condition'].value_counts()

* Preparing x and y data vectors

In [ ]:
### Using MDD,BP dataset
ori_x = ori_df_concat.drop(['condition'], axis=1)
ori_y = ori_df_concat.loc[:,["condition"]]

gen_x = gen_df_concat.drop(['condition'], axis=1)
gen_y = gen_df_concat.loc[:,["condition"]]

In [ ]:
ori_x = ori_x.fillna(ori_x.mean())

In [ ]:
label = ori_y
label = label.replace({'interruption':0})
label = label.replace({'no stress': 1})
label = label.replace({'time pressure': 2})

y_ori = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
label_ = gen_y
label_ = label_.replace({'interruption':0})
label_ = label_.replace({'no stress': 1})
label_ = label_.replace({'time pressure': 2})

y_gen = to_categorical(label_, 3) ## into the format of one-hot encoding

* Separating the test dataset only from original dataframe

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(ori_x, y_ori, test_size = 0.35, random_state = 710674)

* Then concat the generated dataset with training dataset

In [ ]:
x_train_concat = pd.concat([x_trainset, gen_x], ignore_index=True)
y_train_concat = np.concatenate([y_trainset, y_gen])

* Then separating the validation dataset from concat dataframe

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_train_concat, y_train_concat, test_size = 0.3, random_state = 710674)

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
class_weight = {0:1, 1:1, 2:1}

In [ ]:
#### Generate the model
## this model is adequate for full variables usage
model = Sequential()
model.add(Dense(64, input_dim = x_train.shape[1], activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(args.num_classes, activation = 'softmax'))

* optimization function, model compile, and model training

In [ ]:
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-5, momentum = args.momentum)

In [ ]:
# LearningRateScheduler 설정
# lr_scheduler = LearningRateScheduler(lambda epoch: cyclic_learning_rate(epoch, base_lr=0.01, max_lr=0.006, step_size=300))
lr_scheduler = LearningRateScheduler(lambda epoch: exp_range_lr(epoch, base_lr=0.01, max_lr=0.006, step_size=30, gamma=0.99994))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2)   ## using no class weight
# history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, verbose=2, class_weight = class_weight)  ## using class weight
history = model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.bs, callbacks=[lr_scheduler], 
                    validation_data = (x_vali, y_vali), class_weight = class_weight, verbose=2) ## using LR scheduler callback

***

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, model.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

***

## VAE + DNN?

### data preparation

In [ ]:
data = df_reduced.copy()
# data = data.groupby('condition').apply(lambda x: x.sample(300)).reset_index(drop=True)

In [ ]:
data_int = data[data['condition']=="interruption"]
data_nos = data[data['condition']=="no stress"]
data_tip = data[data['condition']=="time pressure"]

In [ ]:
class Args_vaednn:
    # arugments
    epochs=150
    bs=32
    lr=0.0001
    momentum=0.9
    num_classes= 3
    seed=710674

args_vaednn = Args_vaednn()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
data_vae = data.copy()
# data_vae = data_int.copy()
# data_vae = data_nos.copy()
# data_vae = data_tip.copy()

In [ ]:
data_vae.shape

In [ ]:
# y includes our labels and x includes our features
y = data_vae.condition
list = ['condition']
x = data_vae.drop(list, axis = 1 )

In [ ]:
data_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

data_x[:] = scaler.fit_transform(data_x[:])
data_x = data_x.round(decimals=6)

In [ ]:
########## FULL GROUP CLASSIFICATION ##########
label = y
label = label.replace({'interruption':0})
label = label.replace({'no stress': 1})
label = label.replace({'time pressure': 2})

data_y = to_categorical(label, 3) ## into the format of one-hot encoding

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2, random_state = 710674)

### model

In [ ]:
input_dim = data_x.shape[1]
latent_dim = 10

In [ ]:
## encoder model for VAE+DNN
def build_encoder(input_dim, latent_dim):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(32, activation='relu')(x)
    
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)
    
    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.0)
        return z_mean + K.exp(z_log_var) * epsilon

    z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    return Model(inputs, [z_mean, z_log_var, z], name='encoder')

In [ ]:
## decoder model for VAE+DNN
def build_decoder(latent_dim, output_dim):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(32, activation='relu')(latent_inputs)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(output_dim)(x)
    
    return Model(latent_inputs, outputs, name='decoder')

In [ ]:
## VAE model
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        reconstruction_loss = mse(inputs, reconstructed) * input_dim
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        self.add_loss(K.mean(reconstruction_loss + kl_loss))
        return reconstructed

In [ ]:
encoder = build_encoder(input_dim, latent_dim)
decoder = build_decoder(latent_dim, input_dim)

vae = VAE(encoder, decoder)
vae.compile(optimizer=Adam())

In [ ]:
# VAE 모델 학습
vae.fit(x_train, epochs=args_vaednn.epochs, batch_size=args_vaednn.bs, validation_split=0.2)

In [ ]:
## add fully-connected classifier at the back of the VAE model
def build_vae_with_classifier(vae, input_dim):
    vae_input = layers.Input(shape=(input_dim,))
    vae_output = vae(vae_input)  # output from vae (reconstructed dataset)
    
    ## classifier defining
    x = layers.Dense(64, activation='relu')(vae_output)
    x = layers.Dense(32, activation='relu')(x)
    classifier_output = layers.Dense(args_vaednn.num_classes, activation='softmax')(x)
    
    return Model(vae_input, classifier_output)

## classifier 
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-5, momentum = args.momentum)
vae_with_classifier = build_vae_with_classifier(vae, input_dim)
vae_with_classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vae_with_classifier.summary()

In [ ]:
## model training
vae_with_classifier.fit(x_train, y_train, epochs=args_vaednn.epochs, batch_size=args_vaednn.bs)

***

In [ ]:
y_predict = vae_with_classifier.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, vae_with_classifier.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")

## VAE + DNN v2

> This model is adding classifier after the encoder section

### model

In [ ]:
input_dim = data_x.shape[1]
latent_dim = 4

In [ ]:
def build_encoder(input_dim, latent_dim):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dense(8, activation='relu')(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
    return models.Model(inputs, [z_mean, z_log_var, z], name="encoder")

In [ ]:
def build_decoder(latent_dim, output_dim):
    latent_inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(8, activation='relu')(latent_inputs)
    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(output_dim, activation='softmax')(x)
    return models.Model(latent_inputs, outputs, name="decoder")

In [ ]:
def build_vae(encoder, decoder, input_dim):
    inputs = layers.Input(shape=(input_dim,))
    z_mean, z_log_var, z = encoder(inputs)
    reconstructed = decoder(z)

    reconstruction_loss = tf.reduce_mean(losses.binary_crossentropy(inputs, reconstructed))
    kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
    vae_loss = reconstruction_loss + kl_loss

    vae = models.Model(inputs, reconstructed)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')
    return vae

In [ ]:
def build_classifier(latent_dim, num_classes):
    classifier_input = layers.Input(shape=(latent_dim,))
    x = layers.Dense(64, activation='relu')(classifier_input)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(32, activation='relu')(x)
    classifier_output = layers.Dense(args_vaednn.num_classes, activation='softmax')(x)
    
    classifier = models.Model(classifier_input, classifier_output, name='classifier')
    # classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
encoder = build_encoder(input_dim, latent_dim)
decoder = build_decoder(latent_dim, input_dim)
vae = build_vae(encoder, decoder, input_dim)

vae.fit(x_train, epochs=args_vaednn.epochs, batch_size=args_vaednn.bs)

In [ ]:
z_train = encoder.predict(x_train)[2]

classifier = build_classifier(latent_dim, args_vaednn.num_classes)
classifier.fit(z_train, y_train, epochs=args_vaednn.epochs, batch_size=args_vaednn.bs)

In [ ]:
# encoder.predict(x_test)[2]

***

In [ ]:
z_test = encoder.predict(x_test)[2]

In [ ]:
y_predict = classifier.predict(z_test)

In [ ]:
y_predict = np.argmax(y_predict, axis = 1)

In [ ]:
y_test

In [ ]:
z_test = encoder.predict(x_test)[2]

y_predict = classifier.predict(z_test)
y_predict = np.argmax(y_predict, axis = 1)
# y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')
auc = roc_auc_score(y_test, vae_with_classifier.predict(x_test, verbose=0), multi_class='ovr')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("The AUC score is:", round(auc, 4))
print("=============================================")